In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.1 MB/s eta 0:00:00


In [2]:
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB

In [7]:
from datasets import load_dataset
dataset = load_dataset("carblacac/twitter-sentiment-analysis")
text = dataset['train']['text']
feeling = dataset['train']['feeling']

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Creating json from Arrow format:   0%|          | 0/120 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/30 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/62 [00:00<?, ?ba/s]

Generating train split:   0%|          | 0/119988 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/29997 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/61998 [00:00<?, ? examples/s]

In [12]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(text, feeling, test_size=0.2, random_state=3113)
print(len(x_train), len(x_test))

95990 23998


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(TfidfVectorizer(), MultinomialNB())
pipe.fit(x_train, y_train)
y_pred = pipe.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.72      0.84      0.77     11960
           1       0.81      0.68      0.74     12038

    accuracy                           0.76     23998
   macro avg       0.76      0.76      0.76     23998
weighted avg       0.76      0.76      0.76     23998



In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline

# Keep for overnight execution
pipe = make_pipeline(TfidfVectorizer(), SVC())
#pipe.fit(x_train, y_train)
#y_pred = pipe.predict(x_test)
print(classification_report(y_test, y_pred))

KeyboardInterrupt: ignored

In [20]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense

In [21]:
def preprocess(data):
  return {'text':data['text'],'feeling':data['feeling']}
train_data = dataset['train'].map(preprocess)
test_data = dataset['test'].map(preprocess)

Map:   0%|          | 0/119988 [00:00<?, ? examples/s]

Map:   0%|          | 0/61998 [00:00<?, ? examples/s]

In [22]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(train_data['text'])
vocab_size = len(tokenizer.word_index)+1

In [23]:
train_seq = tokenizer.texts_to_sequences(train_data['text'])
test_seq = tokenizer.texts_to_sequences(test_data['text'])

In [24]:
max_len = 32
trainp = tf.keras.preprocessing.sequence.pad_sequences(train_seq, maxlen=max_len)
testp = tf.keras.preprocessing.sequence.pad_sequences(test_seq, maxlen=max_len)

In [25]:
embed_len = 32
model = Sequential([
    Embedding(input_dim=vocab_size,output_dim=embed_len,input_length=max_len),
    Bidirectional(LSTM(64,return_sequences=True)),
    Bidirectional(LSTM(32)),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [26]:
import numpy
y_train = numpy.array(train_data['feeling'])
y_test = numpy.array(test_data['feeling'])
model.fit(trainp, y_train, epochs=4, validation_data=(testp,y_test))

Epoch 1/8
3750/3750 [==============================] - 502s 131ms/step - loss: 0.4755 - accuracy: 0.7717 - val_loss: 0.4426 - val_accuracy: 0.7933
Epoch 2/8
3750/3750 [==============================] - 487s 130ms/step - loss: 0.3090 - accuracy: 0.8687 - val_loss: 0.5138 - val_accuracy: 0.7766
Epoch 3/8
3750/3750 [==============================] - 479s 128ms/step - loss: 0.1927 - accuracy: 0.9242 - val_loss: 0.6023 - val_accuracy: 0.7661
Epoch 4/8
3750/3750 [==============================] - 476s 127ms/step - loss: 0.1311 - accuracy: 0.9473 - val_loss: 0.6747 - val_accuracy: 0.7658
Epoch 5/8
 839/3750 [=====>........................] - ETA: 5:42 - loss: 0.0829 - accuracy: 0.9670

KeyboardInterrupt: ignored